This notebook contains various convergence tests for MPFA.

They are included mainly to complement the tests for MPSA.

In [1]:
import numpy as np
from scipy.sparse.linalg import spsolve
import scipy.sparse as sps
import sympy
from math import pi

import setup_grids

from porepy.numerics.fv import mpfa, fvutils

from porepy.params.tensor import SecondOrderTensor as perm
from porepy.params import bc

In [2]:
# Analytical solution
x, y = sympy.symbols('x y')
u = x * (1-x) * sympy.sin(x) * sympy.cos(y)
u_f = sympy.lambdify((x, y), u, 'numpy')
dux = sympy.diff(u, x)
duy = sympy.diff(u, y)
dux_f = sympy.lambdify((x, y), dux, 'numpy')
duy_f = sympy.lambdify((x, y), duy, 'numpy')
rhs = -sympy.diff(dux, x) - sympy.diff(duy, y)
rhs_f = sympy.lambdify((x, y), rhs, 'numpy')

###
# This is where parameters can be modified to alter the convergence test.
# The remaining lines should 
np.random.seed(42)
base = 4
domain = np.array([1, 1])
basedim = np.array([base, base])
num_refs = 3
grid_type = 'cart'

def run_convergence(grid_type, pert):
    u_err = []
    flux_err = []

    for g in setup_grids.grid_sequence(basedim, num_refs, grid_type, pert, domain=domain):
        # Reset the random seed for every grid realization.
        # This should make no difference for the convergence test, 
        # but it makes sure that we can run unit tests based on the values obtained
        # here.
        np.random.seed(42)
        
        # Permeability tensor
        k = perm(2, np.ones(g.num_cells))

        # Set type of boundary conditions - Dirichlet
        bound_faces = g.tags['domain_boundary_faces'].nonzero()[0]
        bound_cond = bc.BoundaryCondition(g, bound_faces, ['dir'] * bound_faces.size)
        
        # MPFA discretization, and system matrix
        flux, bound_flux, _, _ = mpfa.mpfa(g, k, bound_cond)
        div = fvutils.scalar_divergence(g)
        a = div * flux
        
        # Boundary conditions
        xf = g.face_centers
        u_bound = np.zeros(g.num_faces)
        u_bound[bound_faces] = u_f(xf[0, bound_faces], xf[1, bound_faces])
        
        # Right hand side - contribution from the solution and the boundary conditions
        xc = g.cell_centers
        b = rhs_f(xc[0], xc[1]) * g.cell_volumes - div * bound_flux * u_bound

        # Solve system, derive fluxes
        u_num = spsolve(a, b)
        flux_num = flux * u_num + bound_flux * u_bound

        # Exact solution
        u_ex = u_f(xc[0], xc[1])
        du_ex_faces = np.vstack((dux_f(xf[0], xf[1]), duy_f(xf[0], xf[1])))
        flux_ex = -np.sum(g.face_normals[:2] * du_ex_faces, axis=0)
        flux_diff = flux_num - flux_ex
  
        u_err.append(np.sqrt(np.sum(g.cell_volumes * (u_num - u_ex)**2)) /
                     np.sqrt(np.sum(g.cell_volumes * u_ex**2)))
        flux_err.append(np.sqrt(np.sum((g.face_areas ** g.dim) * flux_diff**2))/
                        np.sqrt(np.sum((g.face_areas ** g.dim) * flux_ex**2)))
    return u_err, flux_err

grids = ['cart', 'triangular']


u_cart_nopert, f_cart_nopert = run_convergence('cart', 0)
print('Cartesian errors - no perturbations')
print(u_cart_nopert,'pressure error')
print(f_cart_nopert,'flux error')

u_cart_pert, f_cart_pert = run_convergence('cart', 0.5)
print('Cartesian errors - perturbation 0.5')
print(u_cart_pert,'pressure error')
print(f_cart_pert,'flux error')


u_triang_nopert, f_triang_nopert = run_convergence('triangular', 0)
print('Triangular errors - no perturbations')
print(u_triang_nopert,'pressure error')
print(f_triang_nopert,'flux error')


u_triang_pert, f_triang_pert = run_convergence('triangular', 0.5)
print('Triangular errors - perturbation 0.5')
print(u_triang_pert,'pressure error')
print(f_triang_pert,'flux error')

# EK: These values were hard-coded 11.06.2017.
assert np.abs(u_cart_nopert[2] - 0.0065122960444455652) < 1e-10
assert np.abs(f_cart_nopert[2] - 0.011371514390600204) < 1e-10
assert np.abs(u_cart_pert[2] - 0.0073837999427839654) < 1e-10
assert np.abs(f_cart_pert[2] - 0.016113648245047234) < 1e-10

# RB: These values were hard-coded 06.09.2018.
assert np.abs(u_triang_nopert[2] - 0.0031466771286353342) < 1e-10
assert np.abs(f_triang_nopert[2] - 0.005705542214407857) < 1e-10
assert np.abs(u_triang_pert[2] - 0.003871569621335528) < 1e-10
assert np.abs(f_triang_pert[2] - 0.011882770637109161) < 1e-10

Cartesian errors - no perturbations
[0.09744623673648027, 0.02563304567729676, 0.006512296044445567] pressure error
[0.11792041665757692, 0.038488831450398144, 0.011371514390600216] flux error
Cartesian errors - perturbation 0.5
[0.10329022796802324, 0.02662299818877489, 0.007383799942783786] pressure error
[0.10417700714847174, 0.049037576768854035, 0.016113648245047286] flux error
Triangular errors - no perturbations
[0.04865380618959045, 0.012528781793590854, 0.0031466771286353342] pressure error
[0.06787911458237876, 0.020085847188375933, 0.005705542214407857] flux error
Triangular errors - perturbation 0.5
[0.05695409168995138, 0.015068959494986567, 0.003871569621335528] pressure error
[0.053652948523950576, 0.031155048071189857, 0.011882770637109161] flux error


# Neumann boundary condition

In [3]:
# Analytical solution
x, y = sympy.symbols('x y')
u = x * (1-x) * sympy.sin(x) * sympy.cos(y)
u_f = sympy.lambdify((x, y), u, 'numpy')
dux = sympy.diff(u, x)
duy = sympy.diff(u, y)
dux_f = sympy.lambdify((x, y), dux, 'numpy')
duy_f = sympy.lambdify((x, y), duy, 'numpy')
rhs = -sympy.diff(dux, x) - sympy.diff(duy, y)
rhs_f = sympy.lambdify((x, y), rhs, 'numpy')

###
# This is where parameters can be modified to alter the convergence test.
# The remaining lines should 
np.random.seed(42)
base = 4
domain = np.array([1, 1])
basedim = np.array([base, base])
num_refs = 3
grid_type = 'cart'

def run_convergence(grid_type, pert):
    u_err = []
    flux_err = []

    for g in setup_grids.grid_sequence(basedim, num_refs, grid_type, pert, domain=domain):
        # Reset the random seed for every grid realization.
        # This should make no difference for the convergence test, 
        # but it makes sure that we can run unit tests based on the values obtained
        # here.
        np.random.seed(42)
        
        # Permeability tensor
        k = perm(2, np.ones(g.num_cells))
        
        # Exact solution
        xf = g.face_centers
        xc = g.cell_centers
        u_ex = u_f(xc[0], xc[1])
        du_ex_faces = np.vstack((dux_f(xf[0], xf[1]), duy_f(xf[0], xf[1])))
        flux_ex = -np.sum(g.face_normals[:2] * du_ex_faces, axis=0)

        # Set type of boundary conditions - Dirichlet
        bound_faces = g.tags['domain_boundary_faces'].nonzero()[0]
        n = g.nodes
        top = bc.face_on_side(g, 'ymax')
        bot = bc.face_on_side(g, 'ymin')
        left = bc.face_on_side(g, 'xmin')
        right = bc.face_on_side(g, 'xmax')

        dir_faces = np.asarray(right[0])
        neu_faces = np.hstack((left[0], bot[0], top[0]))
        bound_cond = bc.BoundaryCondition(g, dir_faces, ['dir'] * len(dir_faces))
                
        # MPFA discretization, and system matrix
        flux, bound_flux, _, _ = mpfa.mpfa(g, k, bound_cond)
        div = fvutils.scalar_divergence(g)
        a = div * flux
        
        # Boundary conditions
        nfi, _, sgn = sps.find(g.cell_faces[neu_faces,:])
        u_bound = np.zeros(g.num_faces)
        u_bound[dir_faces] = u_f(xf[0, dir_faces], xf[1, dir_faces])
        # innflow is always positive, so need to flip flux according to normal direction.
        u_bound[neu_faces[nfi]] = flux_ex[neu_faces[nfi]] * (sgn)

        # Right hand side - contribution from the solution and the boundary conditions
        xc = g.cell_centers
        b = rhs_f(xc[0], xc[1]) * g.cell_volumes - div * bound_flux * u_bound

        # Solve system, derive fluxes
        u_num = spsolve(a, b)
        flux_num = flux * u_num + bound_flux * u_bound

        # Exact solution
        flux_diff = flux_num - flux_ex
  
        u_err.append(np.sqrt(np.sum(g.cell_volumes * (u_num - u_ex)**2)) /
                     np.sqrt(np.sum(g.cell_volumes * u_ex**2)))
        flux_err.append(np.sqrt(np.sum((g.face_areas ** g.dim) * flux_diff**2))/
                        np.sqrt(np.sum((g.face_areas ** g.dim) * flux_ex**2)))
    return u_err, flux_err

grids = ['cart', 'triangular']

## No tests on perturbed grids - I'm too lazy to find faces of the respective boundaries


u_cart_nopert, f_cart_nopert = run_convergence('cart', 0)
u_cart_rate = np.array(u_cart_nopert)
u_cart_rate = u_cart_rate[:-1] / u_cart_rate[1:]
f_cart_rate = np.array(f_cart_nopert)
f_cart_rate = f_cart_rate[:-1] / f_cart_rate[1:]
print('Cartesian errors - no perturbations')
print(u_cart_nopert,'pressure error')
print(u_cart_rate, 'convergence rate u^n-1 / u^n\n')
print(f_cart_nopert,'flux error')
print(f_cart_rate, 'convergence rate f^n-1 / f^n\n')

u_triang_nopert, f_triang_nopert = run_convergence('triangular', 0)
u_triang_rate = np.array(u_triang_nopert)
u_triang_rate = u_triang_rate[:-1] / u_triang_rate[1:]
f_triang_rate = np.array(f_triang_nopert)
f_triang_rate = f_triang_rate[:-1] / f_triang_rate[1:]

print('Triangular errors - no perturbations')
print(u_triang_nopert,'pressure error')
print(u_triang_rate, 'convergence rate u^n-1 / u^n\n')
print(f_triang_nopert,'flux error')
print(f_triang_rate, 'convergence rate f^n-1 / f^n')
# EK: These values were hard-coded 11.06.2017.
assert np.abs(u_cart_nopert[2] - 0.01341819229106214) < 1e-10
assert np.abs(f_cart_nopert[2] - 0.0014594922987503009) < 1e-10

# RB: These values were hard-coded 06.09.2018.
assert np.abs(u_triang_nopert[2] - 0.007020068836548777) < 1e-10
assert np.abs(f_triang_nopert[2] - 0.0033976662367782447) < 1e-10


Cartesian errors - no perturbations
[0.2177630338617197, 0.053854028644456195, 0.013418192291060563] pressure error
[4.04357927 4.01350849] convergence rate u^n-1 / u^n

[0.020449627547754923, 0.00558959841388443, 0.0014594922987497814] flux error
[3.65851463 3.82982385] convergence rate f^n-1 / f^n

Triangular errors - no perturbations
[0.1154963072658415, 0.028265747189434955, 0.007020068836548777] pressure error
[4.08608718 4.02642023] convergence rate u^n-1 / u^n

[0.04223113257437278, 0.011882184577715638, 0.0033976662367782447] flux error
[3.55415558 3.49716062] convergence rate f^n-1 / f^n


# Robin boundary condition

In [4]:
# Analytical solution
x, y = sympy.symbols('x y')
u = x * (1-x) * sympy.sin(x) * sympy.cos(y)
u_f = sympy.lambdify((x, y), u, 'numpy')
dux = sympy.diff(u, x)
duy = sympy.diff(u, y)
dux_f = sympy.lambdify((x, y), dux, 'numpy')
duy_f = sympy.lambdify((x, y), duy, 'numpy')
rhs = -sympy.diff(dux, x) - sympy.diff(duy, y)
rhs_f = sympy.lambdify((x, y), rhs, 'numpy')

###
# This is where parameters can be modified to alter the convergence test.
# The remaining lines should 
np.random.seed(42)
base = 4
domain = np.array([1, 1])
basedim = np.array([base, base])
num_refs = 3
grid_type = 'cart'

def run_convergence(grid_type, pert):
    u_err = []
    flux_err = []

    for g in setup_grids.grid_sequence(basedim, num_refs, grid_type, pert, domain=domain):
        # Reset the random seed for every grid realization.
        # This should make no difference for the convergence test, 
        # but it makes sure that we can run unit tests based on the values obtained
        # here.
        np.random.seed(42)
        
        # Permeability tensor
        k = perm(2, np.ones(g.num_cells))
        alpha = 1.2
        # Exact solution
        xf = g.face_centers
        xc = g.cell_centers
        u_ex = u_f(xc[0], xc[1])
        du_ex_faces = np.vstack((dux_f(xf[0], xf[1]), duy_f(xf[0], xf[1])))
        flux_ex = -np.sum(g.face_normals[:2] * du_ex_faces, axis=0)

        # Set type of boundary conditions - Dirichlet
        bound_faces = g.tags['domain_boundary_faces'].nonzero()[0]
        n = g.nodes
        top = bc.face_on_side(g, 'ymax')
        bot = bc.face_on_side(g, 'ymin')
        left = bc.face_on_side(g, 'xmin')
        right = bc.face_on_side(g, 'xmax')

        dir_faces = np.asarray(left[0])
        rob_faces = np.hstack((right[0]))
        neu_faces = np.hstack((bot[0], top[0]))
        bc_faces = np.hstack([dir_faces, rob_faces])
        bc_names = ['dir']*len(dir_faces) + ['rob'] * len(rob_faces)
        bound_cond = bc.BoundaryCondition(g, bc_faces,  bc_names)
                
        # MPFA discretization, and system matrix
        flux, bound_flux, _, _ = mpfa.mpfa(g, k, bound_cond, alpha=alpha)
        div = fvutils.scalar_divergence(g)
        a = div * flux
        
        # Boundary conditions
        nfi, _, sgn_n = sps.find(g.cell_faces[neu_faces,:])
        rfi, _, sgn = sps.find(g.cell_faces[rob_faces,:])
        u_bound = np.zeros(g.num_faces)
        u_bound[dir_faces] = u_f(xf[0, dir_faces], xf[1, dir_faces])
        # innflow is always negative, so need to flip flux according to normal direction.
        u_bound[neu_faces[nfi]] = flux_ex[neu_faces[nfi]] * (sgn_n)
        u_bound[rob_faces[rfi]] = flux_ex[rob_faces[rfi]] * (sgn) +\
                                  alpha * u_f(xf[0, rob_faces], xf[1, rob_faces]) * g.face_areas[rob_faces]

        # Right hand side - contribution from the solution and the boundary conditions
        xc = g.cell_centers
        b = rhs_f(xc[0], xc[1]) * g.cell_volumes - div * bound_flux * u_bound

        # Solve system, derive fluxes
        u_num = spsolve(a, b)
        flux_num = flux * u_num + bound_flux * u_bound

        # Exact solution
        flux_diff = flux_num - flux_ex
  
        u_err.append(np.sqrt(np.sum(g.cell_volumes * (u_num - u_ex)**2)) /
                     np.sqrt(np.sum(g.cell_volumes * u_ex**2)))
        flux_err.append(np.sqrt(np.sum((g.face_areas ** g.dim) * flux_diff**2))/
                        np.sqrt(np.sum((g.face_areas ** g.dim) * flux_ex**2)))
    return u_err, flux_err

grids = ['cart', 'triangular']

## No tests on perturbed grids - I'm too lazy to find faces of the respective boundaries


u_cart_nopert, f_cart_nopert = run_convergence('cart', 0)
u_cart_rate = np.array(u_cart_nopert)
u_cart_rate = u_cart_rate[:-1] / u_cart_rate[1:]
f_cart_rate = np.array(f_cart_nopert)
f_cart_rate = f_cart_rate[:-1] / f_cart_rate[1:]
print('Cartesian errors - no perturbations')
print(u_cart_nopert,'pressure error')
print(u_cart_rate, 'convergence rate u^n-1 / u^n\n')
print(f_cart_nopert,'flux error')
print(f_cart_rate, 'convergence rate f^n-1 / f^n\n')

u_triang_nopert, f_triang_nopert = run_convergence('triangular', 0)
u_triang_rate = np.array(u_triang_nopert)
u_triang_rate = u_triang_rate[:-1] / u_triang_rate[1:]
f_triang_rate = np.array(f_triang_nopert)
f_triang_rate = f_triang_rate[:-1] / f_triang_rate[1:]

print('Triangular errors - no perturbations')
print(u_triang_nopert,'pressure error')
print(u_triang_rate, 'convergence rate u^n-1 / u^n\n')
print(f_triang_nopert,'flux error')
print(f_triang_rate, 'convergence rate f^n-1 / f^n')

# RB: These values were hard-coded 11.09.2018.
assert np.abs(u_cart_nopert[2] - 0.034984342689359696) < 1e-10
assert np.abs(f_cart_nopert[2] - 0.01641158802403413) < 1e-10
assert np.abs(u_triang_nopert[2] - 0.005995729167675974) < 1e-10
assert np.abs(f_triang_nopert[2] - 0.002734187490448117) < 1e-10


Cartesian errors - no perturbations
[0.562515011909662, 0.14015471392164033, 0.034984342689359696] pressure error
[4.01352902 4.00621258] convergence rate u^n-1 / u^n

[0.22376060444435023, 0.061834410597352465, 0.01641158802403413] flux error
[3.61870684 3.76772866] convergence rate f^n-1 / f^n

Triangular errors - no perturbations
[0.11192090940414492, 0.02516849817177693, 0.005995729167675974] pressure error
[4.44686483 4.19773767] convergence rate u^n-1 / u^n

[0.03311849158106908, 0.00866805360137708, 0.002734187490448117] flux error
[3.82075297 3.17024843] convergence rate f^n-1 / f^n


# Heterogeneous permeability field
Jump in upper right corner

In [5]:
# Analytical solution

kappa_list = [1e-6, 1, 1e6]

### End of parameter definitions

# Permeability tensor, scalar for simplicity

def chi(x, y):
    return np.logical_and(np.greater(x, 0.5), np.greater(y, 0.5))

# Analytical solution
x, y = sympy.symbols('x y')
u = sympy.sin(2 * x * pi) * sympy.sin(2 * y * pi) 
u_f = sympy.lambdify((x, y), u, 'numpy')
dux = sympy.diff(u, x)
duy = sympy.diff(u, y)
dux_f = sympy.lambdify((x, y), dux, 'numpy')
duy_f = sympy.lambdify((x, y), duy, 'numpy')
rhs = -sympy.diff(dux, x) - sympy.diff(duy, y)
rhs_f = sympy.lambdify((x, y), rhs, 'numpy')


###
# This is where parameters can be modified to alter the convergence test.
# The remaining lines should 
np.random.seed(42)
base = 4
domain = np.array([1, 1])
basedim = np.array([base, base])
num_refs = 3
grid_type = 'cart'

def run_convergence(grid_type, pert):
    u_err = np.zeros((num_refs, len(kappa_list)))
    flux_err = np.copy(u_err)
    for iter1, g in enumerate(setup_grids.grid_sequence_fixed_lines(basedim, num_refs, grid_type, pert, subdom_func=chi)):
        # Reset the random seed for every grid realization.
        # This should make no difference for the convergence test, 
        # but it makes sure that we can run unit tests based on the values obtained
        # here.
        np.random.seed(42)
        
        for iter2, kappa in enumerate(kappa_list):
            char_func_cells = chi(g.cell_centers[0], g.cell_centers[1]) * 1.
            perm_vec = (1 - char_func_cells) + kappa * char_func_cells
            # Permeability tensor
            k = perm(2, perm_vec)
            bound_faces = g.tags['domain_boundary_faces'].nonzero()[0]
            bound_cond = bc.BoundaryCondition(g, bound_faces, ['dir'] * bound_faces.size)
            flux, bound_flux, _, _ = mpfa.mpfa(g, k, bound_cond)

            xc = g.cell_centers
            xf = g.face_centers
            char_func_bound = chi(xf[0, bound_faces], xf[1, bound_faces]) * 1
            
            u_bound = np.zeros(g.num_faces)
            u_bound[bound_faces] = u_f(xf[0, bound_faces], xf[1, bound_faces]) \
                                / ((1 - char_func_bound) + kappa * char_func_bound)

            div = fvutils.scalar_divergence(g)
            a = div * flux

            b = rhs_f(xc[0], xc[1]) * g.cell_volumes - div * bound_flux * u_bound

            u_num = spsolve(a, b)
            u_ex = u_f(xc[0], xc[1]) / ((1 - char_func_cells) + kappa * char_func_cells)
            
            flux_num = flux * u_num + bound_flux * u_bound
            du_ex_faces = np.vstack((dux_f(xf[0], xf[1]), duy_f(xf[0], xf[1])))
            flux_ex = -np.sum(g.face_normals[:2] * du_ex_faces, axis=0)
            flux_diff = flux_num - flux_ex

            u_err[iter1, iter2] = (np.sqrt(np.sum(g.cell_volumes * (u_num - u_ex)**2)) /
                         np.sqrt(np.sum(g.cell_volumes * u_ex**2)))
            flux_err[iter1, iter2] = (np.sqrt(np.sum((g.face_areas ** g.dim) * flux_diff**2))/
                            np.sqrt(np.sum((g.face_areas ** g.dim) * flux_ex**2)))

    return u_err, flux_err

grids = ['cart', 'triangular']

## No tests on perturbed grids - I'm too lazy to find faces of the respective boundaries

u_cart_nopert, f_cart_nopert = run_convergence('cart', 0)
print('Cartesian errors - no perturbations')
print(u_cart_nopert,'pressure error')
print(f_cart_nopert,'flux error')

u_cart_pert, f_cart_pert = run_convergence('cart', 0.5)
print('Cartesian errors - perturbation 0.5')
print(u_cart_pert,'pressure error')
print(f_cart_pert,'flux error')


u_triang_nopert, f_triang_nopert = run_convergence('triangular', 0)
print('Triangular errors - no perturbations')
print(u_triang_nopert,'pressure error')
print(f_triang_nopert,'flux error')


u_triang_pert, f_triang_pert = run_convergence('triangular', 0.5)
print('Triangular errors - perturbation 0.5')
print(u_triang_pert,'pressure error')
print(f_triang_pert,'flux error')

# EK: These values were hard-coded 11.06.2017.
assert np.abs(u_cart_nopert[2, 0] - 0.01295075) < 1e-7
assert np.abs(f_cart_nopert[2, 0] - 0.00645454) < 1e-7
assert np.abs(u_cart_pert[2, 0] - 0.01484212) < 1e-7
assert np.abs(f_cart_pert[2, 0] - 0.01687373) < 1e-7

# RB: These values were hard-coded 06.09.2018.
assert np.abs(u_triang_nopert[2, 0] - 0.01250799) < 1e-7
assert np.abs(f_triang_nopert[2, 0] - 0.00653148) < 1e-7
assert np.abs(u_triang_pert[2, 0] - 0.01520087) < 1e-7
assert np.abs(f_triang_pert[2, 0] - 0.01302646) < 1e-7


Cartesian errors - no perturbations
[[0.23370055 0.23370055 0.23370055]
 [0.05302929 0.05302929 0.05302929]
 [0.01295075 0.01295075 0.01295075]] pressure error
[[0.11072073 0.11072073 0.11072073]
 [0.02617215 0.02617215 0.02617215]
 [0.00645454 0.00645454 0.00645454]] flux error
Cartesian errors - perturbation 0.5
[[0.28144397 0.3116494  0.28115807]
 [0.08134885 0.0741203  0.06868011]
 [0.01484212 0.01673688 0.01728718]] pressure error
[[0.18233422 0.18698603 0.20433553]
 [0.05353656 0.05186143 0.05246204]
 [0.01687373 0.01673338 0.01674571]] flux error
Triangular errors - no perturbations
[[0.18810597 0.19650285 0.19475267]
 [0.0490365  0.04950331 0.0494196 ]
 [0.01250799 0.01255227 0.01253359]] pressure error
[[0.09610297 0.0942635  0.09702186]
 [0.02395548 0.02417054 0.02485514]
 [0.00653148 0.00662373 0.00680908]] flux error
Triangular errors - perturbation 0.5
[[0.25769202 0.30829373 0.30552852]
 [0.08177972 0.07242441 0.06817465]
 [0.01520087 0.01618674 0.01645769]] pressure erro

# 3d test cases

In [6]:
# Analytical solution
x, y, z = sympy.symbols('x y z')
u = sympy.cos(x) * sympy.sin(z) * sympy.cosh(y)
u_f = sympy.lambdify((x, y, z), u, 'numpy')
dux = sympy.diff(u, x)
duy = sympy.diff(u, y)
duz = sympy.diff(u, z)
dux_f = sympy.lambdify((x, y, z), dux, 'numpy')
duy_f = sympy.lambdify((x, y, z), duy, 'numpy')
duz_f = sympy.lambdify((x, y, z), duz, 'numpy')
rhs = -sympy.diff(dux, x) - sympy.diff(duy, y) - sympy.diff(duz, z)
rhs_f = sympy.lambdify((x, y, z), rhs, 'numpy')

###
# This is where parameters can be modified to alter the convergence test.
# The remaining lines should 
np.random.seed(42)
base = 4
domain = np.array([1, 1, 1])
basedim = np.array([base, base, base])
num_refs = 3

def run_convergence(grid_type, pert):
    u_err = []
    flux_err = []

    for g in setup_grids.grid_sequence(basedim, num_refs, grid_type, pert, domain=domain):
        # Reset the random seed for every grid realization.
        # This should make no difference for the convergence test, 
        # but it makes sure that we can run unit tests based on the values obtained
        # here.
        np.random.seed(42)
        
        # Permeability tensor
        k = perm(3, np.ones(g.num_cells))

        # Set type of boundary conditions - Dirichlet
        bound_faces = g.tags['domain_boundary_faces'].nonzero()[0]
        bound_cond = bc.BoundaryCondition(g, bound_faces, ['dir'] * bound_faces.size)
        
        # MPFA discretization, and system matrix
        flux, bound_flux ,_,_= mpfa.mpfa(g, k, bound_cond)
        div = fvutils.scalar_divergence(g)
        a = div * flux

        # Boundary conditions
        xf = g.face_centers
        u_bound = np.zeros(g.num_faces)
        u_bound[bound_faces] = u_f(xf[0, bound_faces], xf[1, bound_faces], xf[2, bound_faces])
        
        # Right hand side - contribution from the solution and the boundary conditions
        xc = g.cell_centers
        b = rhs_f(xc[0], xc[1], xc[2]) * g.cell_volumes - div * bound_flux * u_bound

        # Solve system, derive fluxes
        u_num = spsolve(a, b)
        flux_num = flux * u_num + bound_flux * u_bound

        # Exact solution
        u_ex = u_f(xc[0], xc[1], xc[2])
        du_ex_faces = np.vstack((dux_f(xf[0], xf[1], xf[2]),
                                 duy_f(xf[0], xf[1], xf[2]),
                                 duz_f(xf[0], xf[1], xf[2])))
        flux_ex = -np.sum(g.face_normals * du_ex_faces, axis=0)
        flux_diff = flux_num - flux_ex

        u_err.append(np.sqrt(np.sum(g.cell_volumes * (u_num - u_ex)**2)) /
                     np.sqrt(np.sum(g.cell_volumes * u_ex**2)))
        flux_err.append(np.sqrt(np.sum((g.face_areas ** g.dim) * flux_diff**2))/
                        np.sqrt(np.sum((g.face_areas ** g.dim) * flux_ex**2)))
    return u_err, flux_err

u_cart_nopert, f_cart_nopert = run_convergence('cart', 0)
print('Cartesian errors - no perturbations')
print(u_cart_nopert,'pressure error')
print(f_cart_nopert,'flux error')

u_cart_pert, f_cart_pert = run_convergence('cart', 0.5)
print('Cartesian errors - perturbation 0.5')
print(u_cart_pert,'pressure error')
print(f_cart_pert,'flux error')


u_triang_nopert, f_triang_nopert = run_convergence('tetrahedral', 0)
print('Tetrehedral errors - no perturbations')
print(u_triang_nopert,'pressure error')
print(f_triang_nopert,'flux error')

u_triang_pert, f_triang_pert = run_convergence('tetrahedral', 0.3)
print('Tetrahedral errors - perturbation 0.3')
print(u_triang_pert,'pressure error')
print(f_triang_pert,'flux error')

# EK: These values were hard-coded 11.06.2017.
assert np.abs(u_cart_nopert[2] -0.00030564723918128137 ) < 1e-10
assert np.abs(f_cart_nopert[2] - 0.0028725567629342458) < 1e-10
assert np.abs(u_cart_pert[2] - 0.00032732824057441688) < 1e-10
assert np.abs(f_cart_pert[2] - 0.0041613784496996173) < 1e-10

# RB: These values were hard-coded 06.09.2018.
assert np.abs(u_triang_nopert[2] - 0.00012116396539578085) < 1e-10
assert np.abs(f_triang_nopert[2] - 0.0036882937583153238) < 1e-10
assert np.abs(u_triang_pert[2] - 0.00013484254795268383) < 1e-10
assert np.abs(f_triang_pert[2] - 0.004055069442646703) < 1e-10

Cartesian errors - no perturbations
[0.0038211823411524516, 0.0011419336515988462, 0.00030564723918128137] pressure error
[0.03261861955556977, 0.010014070049802667, 0.002872556762934246] flux error
Cartesian errors - perturbation 0.5
[0.004190264016265817, 0.0012081169906913614, 0.0003273282405744348] pressure error
[0.028005576371853302, 0.01068845032035332, 0.004161378449699483] flux error
Tetrehedral errors - no perturbations
[0.0019052595279009587, 0.00048473242101445527, 0.00012116396539578085] pressure error
[0.01802429803935019, 0.007938868913250801, 0.0036882937583153238] flux error
Tetrahedral errors - perturbation 0.3
[0.002253246419480656, 0.0005353463651070588, 0.00013484254795268383] pressure error
[0.019027005210865585, 0.009029346065771729, 0.004055069442646703] flux error
